# Compute conformer energies for a small molecule

This notebook illustrates reading conformers of a molecule from an SDF file and computation of vacuum conformer energies using a SMIRNOFF force field.

In [1]:
# Load in the molecule and its conformers.
# You would start here if you had your own SDF input files.
# Note that all conformers of the same molecule are loaded as separate Molecule objects
from openforcefield.topology import Molecule
#loaded_molecules = Molecule.from_file('imatinib_conformers.sdf')
loaded_molecules = Molecule.from_file('ruxolitinib_conformers.sdf')
# Collatate all conformers of the same molecule
# NOTE: This isn't necessary if you have already loaded or created multi-conformer molecules;
# it is just needed because our SDF reader does not automatically collapse conformers.
molecules = [loaded_molecules[0]]
for molecule in loaded_molecules[1:]:
    if molecule == molecules[-1]:
        for conformer in molecule.conformers:
            molecules[-1].add_conformer(conformer)
    else:
        molecules.append(molecule)

n_molecules = len(molecules)
n_conformers = sum([mol.n_conformers for mol in molecules])
print(f'{n_molecules} unique molecule(s) loaded, with {n_conformers} total conformers')

_ColormakerRegistry()

1 unique molecule(s) loaded, with 10 total conformers


In [2]:
# Load the openff-1.0.0 force field appropriate for vacuum calculations (without constraints)
from openforcefield.typing.engines.smirnoff import ForceField
forcefield = ForceField('openff_unconstrained-1.1.0.offxml')

In [3]:
# Loop over molecules and compute energies of each conformer
for molecule in molecules:
    print('%s : %d conformers' % (molecule.name, molecule.n_conformers))
    # Create an OpenMM System for the small molecule in vacuum
    system = forcefield.create_openmm_system(molecule.to_topology())
    # Compute energy for all conformers
    from simtk import openmm, unit
    integrator = openmm.VerletIntegrator(1*unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('Reference')
    context = openmm.Context(system, integrator, platform)
    for conformer_index, conformer in enumerate(molecule.conformers):
        context.setPositions(conformer)
        potential = context.getState(getEnergy=True).getPotentialEnergy()
        print('Conformer %5d / %5d : %8.3f kcal/mol' % (conformer_index, molecule.n_conformers, potential/unit.kilocalories_per_mole))
    # Clean up OpenMM Context
    del context, integrator

ruxolitinib : 10 conformers
Conformer     0 /    10 :   34.921 kcal/mol
Conformer     1 /    10 :   43.903 kcal/mol
Conformer     2 /    10 :   79.537 kcal/mol
Conformer     3 /    10 :   40.710 kcal/mol
Conformer     4 /    10 :   41.273 kcal/mol
Conformer     5 /    10 :   43.646 kcal/mol
Conformer     6 /    10 :   42.257 kcal/mol
Conformer     7 /    10 :   32.198 kcal/mol
Conformer     8 /    10 :   38.401 kcal/mol
Conformer     9 /    10 :   41.009 kcal/mol
